In [54]:
# import des constantes necessaires
from scipy.constants import Avogadro, c, m_e, m_p
import numpy as np # get down babyyyyy
import matplotlib.pyplot as plt # beaux graphiques stylé

# Q1
Exprimez la densité électronique $n_e$ d’un milieu en fonction de sa composition atomique et de sa masse volumique $\rho$, et calculer $n_e$ pour l’eau (liquide) et l’os compact (définition de l’ICRU). On s’appuiera sur les données du NIST pour les compositions atomiques de ces matériaux.

In [58]:
# fonction pour le calcul de n_e, source : chatgpt
n_e = lambda atomic_numbers, fractions_by_weight, rho, A: rho * Avogadro * np.sum(atomic_numbers * fractions_by_weight/A)

rho_bone = 1.85 #g/cm^3
atomic_numbers_bone = np.array([1, 6, 7, 8, 12, 15, 16, 20])
fractions_by_weight_bone = np.array([0.063984, 0.278000, 0.027000, 0.410016, 0.002000, 0.070000, 0.002000, 0.147000])
A_bone = np.array([1.008, 12.011, 14.007, 15.999, 24.305, 30.974, 32.065, 40.078])
I_bone = 91.9 # eV

n_e_bone = n_e(atomic_numbers_bone, fractions_by_weight_bone, rho_bone, A_bone)

rho_water = 1
I_water = 75 # eV
A_water = np.array([1.008, 15.999])
atomic_numbers_water = np.array([1, 8])
fractions_by_weight_water = np.array([0.111894, 0.888106])

n_e_water = n_e(atomic_numbers_water, fractions_by_weight_water, rho_water, A_water)

# Q2

In [57]:
gamma = lambda T: T/(m_p*c**2) + 1
def S_col(T, ):pass

In [52]:
n_e_water

3.342810286137214e+23

In [53]:
n_e_bone

5.905875865014682e+23

In [48]:
m_p

1.67262192369e-27

In [49]:
m_e

9.1093837015e-31